In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

rng = np.random.default_rng()

In [3]:
from mtt.sensor import Sensor
from mtt.simulator import Simulator
from mtt.data import OnlineDataset

init_simulator = lambda: Simulator(
    max_targets=10,
    p_initial=4,
    p_birth=2,
    p_survival=0.95,
    sigma_motion=0.1,
    sigma_initial_state=(3.0, 1.0, 1.0),
    max_distance=1e6,
)
init_sensor = lambda: Sensor(position=(1, 1), noise=(0.2, 0.1), p_detection=0.9)
dataset = OnlineDataset(n_steps=100, length=20, img_size=128, init_simulator=init_simulator, init_sensor=init_sensor)

In [3]:
from mtt.models import Conv3dEncoderDecoder
from torchinfo import summary

model = Conv3dEncoderDecoder(n_channels=64, img_size=128, n_encoder=3, n_hidden=1, kernel_spatial=9, padding_spatial=4)
summary(model, (1,1) + model.input_shape)

Layer (type:depth-idx)                   Output Shape              Param #
Conv3dEncoderDecoder                     --                        --
├─Sequential: 1-1                        [1, 64, 20, 17, 17]       --
│    └─Conv3d: 2-1                       [1, 64, 20, 65, 65]       12,352
│    └─LeakyReLU: 2-2                    [1, 64, 20, 65, 65]       --
│    └─Conv3d: 2-3                       [1, 64, 20, 33, 33]       786,496
│    └─LeakyReLU: 2-4                    [1, 64, 20, 33, 33]       --
│    └─Conv3d: 2-5                       [1, 64, 20, 17, 17]       786,496
│    └─LeakyReLU: 2-6                    [1, 64, 20, 17, 17]       --
├─Sequential: 1-2                        [64, 5780]                --
│    └─Linear: 2-7                       [64, 5780]                33,414,180
│    └─ReLU: 2-8                         [64, 5780]                --
├─Sequential: 1-3                        [1, 1, 20, 128, 128]      --
│    └─ConvTranspose3d: 2-9              [1, 64, 20, 33, 33]   

In [15]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset,
    batch_size=16,
    num_workers=16,
    pin_memory=True,
)
trainer = pl.Trainer(max_epochs=1, gpus=1, profiler="simple")
trainer.fit(model, dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 520 K 
1 | decoder | Sequential | 520 K 
2 | hidden  | Sequential | 0     
---------------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.161     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

FIT Profiler Report

Action                             	|  Mean duration (s)	|Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------
Total                              	|  -              	|_              	|  18.561         	|  100 %          	|
--------------------------------------------------------------------------------------------------------------------------------------
run_training_epoch                 	|  18.529         	|1              	|  18.529         	|  99.828         	|
run_training_batch                 	|  0.15175        	|112            	|  16.996         	|  91.57          	|
get_train_batch                    	|  0.0034679      	|113            	|  0.39188        	|  2.1113         	|
fetch_next_train_batch             	|  0.0034552      	|113            	|  0.39043        	|  2.1035         	|
optimizer_step_with_closure_0      